In [1]:
import sys, csv
import threading, logging, time
import datetime
import json
import kafka
from kafka import KafkaProducer 
import urllib
import requests
import json
import ast
import pandas as pd

### Kod do wysylania danych do serwera AWS

In [ ]:
producer = KafkaProducer(bootstrap_servers=['ec2-3-73-51-46.eu-central-1.compute.amazonaws.com'])
api = "https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e-927d-4ad3-9500-4ab9e55deb59&apikey=c5f14341-39d5-445a-b11b-1c41288cc8a1&type=1&line=221"
data = [1,2]
for d in data: 
    
    while d > 0:
        response = requests.get(api)
        data = response.json()
        df = pd.DataFrame.from_dict(data['result'])
        for row in df.itertuples(index=False):
            producer.send('BUS__221',json.dumps(row).encode('utf-8')).get(timeout=30)
        time.sleep(12)

### Pobranie oraz przygotowanie danych statycznych dot. przystankow

#### Zapisanie danych z API w formacie .json do DataFrame

In [ ]:
response_2 = requests.get("https://api.um.warszawa.pl/api/action/dbstore_get/?id=1c08a38c-ae09-46d2-8926-4f9d25cb0630&apikey=c5f14341-39d5-445a-b11b-1c41288cc8a1")
read_content = response_2.json()
i=0
df_stops = pd.DataFrame(columns=
                        ['Zespol','Slupek','Nazwa_zespolu','ID_ULICY'
                         ,'SZER_GEO','DLUG_GEO','KIERUNEK','OBOWIAZUJACE_OD'])
result = read_content['result']
for values in result:
    data_values = values['values']
    list= []
    for data_fin in data_values:
        list.append(data_fin['value'])
    print(list)
    df_stops.loc[i]=list
    i += 1
    print(i)

#### Zapisanie do pliku csv

In [ ]:
f_stops.to_csv('/Users/mateuszjuzyszyn/Desktop/PRACA DYPLOMOWA/stops.csv',encoding='utf-8')

### 
### 
### 
### 
### 
### Pobranie oraz przygotowanie danych dot. rozkladu jazdy linii 221

#### Ograniczenie do przystankow danej linii

In [ ]:
list =["6039","6089","6090","6197","6198","6212","6215","6221"]
df_stops_221 = df_stops[df_stops.Zespol.isin(list)]

#### Zapisanie do pliku CSV

In [ ]:
points_221 = pd.DataFrame([df_stops_221.SZER_GEO,df_stops_221.DLUG_GEO
                           ,df_stops_221.Zespol,df_stops_221.Slupek
                           ,df_stops_221.Nazwa_zespolu, df_stops_221.KIERUNEK]).transpose()
points_221.to_csv('/Users/mateuszjuzyszyn/Desktop/PRACA_DYPLOMOWA/point_221.csv',encoding='utf-8', header = True)

#### Pętla do zebrania danch odnośnie rozkładu jazdy na wszystkich przystankach

In [ ]:
df_timetable221 = pd.DataFrame(columns=['Line','zespol','Slupek','Symbol_2'
                                        ,'Symbol_1','Brygada','kierunek','trasa','czas'])
for stopID, stopNr in zip(df_stops_221.Zespol, df_stops_221.Slupek):
    busstopNr = stopNr
    busstopId = stopID
    line = '221'
    apikey = 'c5f14341-39d5-445a-b11b-1c41288cc8a1'
    api = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId={}&busstopNr={}&line={}&apikey={}'
    api_fin = api.format(busstopId,busstopNr,line,apikey)
    response_221 = requests.get(api_fin)
    read_content = response_221.json()
    i=0 + len(df_timetable221.index)
    result = read_content['result']
    for values in result:
        data_values = values['values']
        list= [line,busstopId,busstopNr]
        for data_fin in data_values:
            list.append(data_fin['value'])
        print(list)
        df_timetable221.loc[i]=list
        i += 1
        print(i)

#### Zapisanie do pliku csv

In [ ]:
df_timetable221.to_csv('/Users/mateuszjuzyszyn/Desktop/PRACA_DYPLOMOWA/timetable_221.csv',encoding='utf-8', header=True)